## Databases from custom searches

### NCBI 
- Search for venom proteins across all animals. The venom proteins of spiders and snakes have been identified before in *C. fleckeri*
    - venom[All Fields] AND swissprot[filter] AND eukaryotes[filter] AND animals
- Search for families of animal venom proteins
    - `"hemolysin"[All Fields] OR "pore-forming"[All Fields] OR "hemolysis"[All Fields] OR "pore-forming protein"[All Fields] OR ("metalloproteinase"[All Fields] AND "venom"[All Fields]) AND animals[filter]`
- Cnidarian proteins, from INSDC, PDB, RefSeq, PIR and PRF databases

### UniProt
- [Taxonomy search "cnidarian"](https://www.uniprot.org/uniprotkb?query=(taxonomy_id:6073))
    - 309,513 results as of Mon 17 Apr, 2023, 
- Annotated venom and toxin proteins from the [Animal toxin annotation project](https://www.uniprot.org/uniprotkb?query=%28taxonomy_id%3A33208%29%20AND%20%28%28cc_tissue_specificity%3Avenom%29%20OR%20%28keyword%3AKW-0800%29%29%20AND%20%28reviewed%3Atrue%29)
    - 7,736 results as of Mon 17 Apr, 2023




## De novo transcriptome assembly

### Sources
- *C. fleckeri* tentacle transcriptome (data used by @brinkmanTranscriptomeVenomProteom2015): https://www.ncbi.nlm.nih.gov/sra/?term=txid45396[Organism:noexp]
    - Dataset acquired with SRA toolkit: `prefetch SRX891607`
- *C. yamaguchii* tentacle transcriptome: https://www.ncbi.nlm.nih.gov/sra/SRX4928706[accn]
    - Dataset acquired with SRA toolkit: `prefetch SRR8101946`

Files were downloaded in `.sra` format, and were converted into `.fastq` using the `fasterq-dump` tool



#### Details of RNA-Seq

##### *C. fleckeri* tentacles
- Instrument: Illumina HiSeq 2000
- Selection: unspecified
- Layout: PAIRED

**Design:** Tentacles were excised and immediately preserved in RNAlater (Life Technologies). Total RNA was isolated from the C. fleckeri tentacles of a single specimen using a handheld rotor-stator homogenizer (QIAgen) and TRIzol (Life Technologies), according to the manufacturer’s instructions. The RNA was DNase-treated, purified using a Nucleospin RNA II kit (Macherey-Nagel) and eluted in nuclease-free water. RNA purity, concentration and integrity were evaluated using a NanoDrop 2000 UV-Vis spectrophotometer and an Agilent 2100 Bioanalyzer with a RNA 6000 Pico kit. Purified RNA (5 μg; RIN 9.1) was submitted to the Ramaciotti Centre for Genomics (Sydney, Australia) for RNA-seq library construction and 100 bp paired-end sequencing on an Illumina HiSeq2000 sequencer.

##### *C. yamaguchii* tentacles
- Instrument: Illumina HiSeq 2500
- Selection: RANDOM
- Layout: PAIRED

**Design:** Extracted RNA with Illumina TruSeq Stranded mRNA Sample Prep kit



### Initial quality checks
Carried out with `fastqc` and aggregated with `multiqc`
- `fastqc <filename>.fastq -o outputdirectory`
    - Since we are dealing with paired end reads, `fastqc` will analyse the two .fastq files for each sample separately
- (In directory with `fastqc` reports) `multiqc .`
- `fasterq-dump` split the paired-end sra data into forward and reverse reads

#### C. fleckeri
- Per base sequence quality: passed, no median quality scores do not fall below FastQC threshold
- Per sequence quality: passed, no spikes of low quality sequences
- Per base sequence content: failed (rarely ever passes with RNA-seq)
    - FastQC expects parallel lines i.e. a uniform distribution of bases across the read length. The base distribution across the first few bases are nonrandom (e.g. 50% of sequences have C as the first base), but this is to do with RNA-seq library prep methods
    - No need to trim the first 10 bases
- Overrepresented sequences: detected, but no hits against adapters

#### C. yamaguchii
- Per base sequence quality: passed, no median quality scores do not fall below FastQC threshold
- Per sequence quality: passed, no spikes of low quality sequences
- Per base sequence content: failed (rarely ever passes with RNA-seq)
    - Same protocol as above
- Overrepresented sequences: none

###### References
[Dr Simon Cockell's youtube channel](https://www.youtube.com/watch?v=FhyOYN_PWh8)



### Quality control and filtering


#### Adapter trimming with `fastp`
```
fastp -i <forwardRead> -I <reverseRead> --detect_adapter_for_pe -c -o <forwardoutput> -O <reverseoutput> -l 50
```
**Flag** info
- `-Q` disables quality filtering (for now)
- Adapters and poly A are automatically trimmed (even for PE data) and `fastp`
    - `--detect_adapter_for_pe` tells `fastp` to look for adapters in PE data
- `-c` enables overlap analysis for PE data, where `fastp` tries to correct mismatched base pairs between overlapping regions (prioritizing the base with the higher quality)
    - Adjustable parameters
        - `overlap_len_require` (default is 30)
        - `overlap_diff_limit` (default is 5)
        - `overlap_diff_percent_limit` (default is 20%)
- `-l` Sets the length that determines which reads are discarded (anything shorter than -l). Chose 50

**Code used**
```{bash}
fastp -i ~/data/raw/Cf_RNA/Cflec_1.fastq -I ~/data/raw/Cf_RNA/Cflec_2.fastq --detect_adapter_for_pe -c -o ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf1_trim.fastq.gz -O ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf2_trim.fastq.gz -l 50

fastp -i ~/data/raw/Cy_RNA/Cyama_1.fastq -I ~/data/raw/Cy_RNA/Cyama_2.fastq --detect_adapter_for_pe -c -o ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy1_trim.fastq.gz -O ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy2_trim.fastq.gz -l 50
```

Report can be found in `~/data/RNA-seq/1-adapters_trimmed/`
- 96% and 98.9% of Cf and Cy reads passed the filter respectively

In [ ]:
# Stats summary
seqkit stats $(find ~/data/raw/ -name "*fastq*") $(find ~/data/RNA-seq/1-adapters_trimmed -name "*fastq*") > raw_vs_adapters_filtered.txt

#### Filtering ncRNA with `bbduk`
- Although RNA-seq library preparation enriches mRNA, there may still be some ncRNAs (especially rRNAs) present in the sample (need further citation for this)
- These can be filtered using a database of ncRNAs as a reference 
    - Used a custom ncRNA database acquired from [RNA central](https://rnacentral.org/) with search `cnidaria AND so_rna_type_name:"NcRNA"` (47,056 seqs) Sat 22 Apr, 2023 
    - The chosen tool is `bbduk`, which filters reads matching reference k-mers (generated from the database described above)
        - To err on the side of caution and to reduce the chance of false positive matches (mRNA matching with ncRNA), will choose a k of 31 (the maximum k that `bbduk` supports)
- An initial search with k = 31 filtered out 53.6 and 12.0% of the Cf and Cy reads respectively, with the most abundant ncRNA reads being rRNA as expected
- Due to the rather high number of sequences filtered, will repeat the process but with two smaller databases: 
    - 1) only Cubozoan ncRNAs and 
    - 2) Only Chironex rRNA or mitochondrial sequences 
        - NCBI search: "Chironex" AND ("rRNA" OR "mitochondrial") 
        - RNACentral: (chironex* AND so_rna_type_name:"RRNA") AND entry_type:"Sequence" Sat 22 Apr, 2023 
- Because of the large number of matches even with the smalleset database (Chironex seqs only), you should BLAST the overrepresented reads (from the initial checks with fastqc) just to see what is going on 
    - `blastn –db nt –query nt.fsa –out results.out`, where nt.fsa a FASTA file of the nucleotides you want to check against the database
- Exact command
- `blastn -db ~/data/reference/unwanted_seqs/Chironex_BLASTDB/Chironex_DB -query <query> -out <results>` 

In [ ]:
# Command structure
bbduk.sh in1= in2= out1= out2= ref=<database> k=31 outm=detected_ncRNAs.fastq.gz stats=<report file>
# k-mer filtering mode is the default

In [ ]:
    # All Cnidarian search

# C. fleckeri
cd ~/data/RNA-seq/2-rRNA_filter/; bbduk.sh in1=~/data/RNA-seq/1-adapters_trimmed/Cf/Cf1_trim.fastq.gz in2=~/data/RNA-seq/1-adapters_trimmed/Cf/Cf2_trim.fastq.gz out1=All_Cnidaria/Cf/clean_Cf1.fastq.gz out2=All_Cnidaria/Cf/clean_Cf2.fastq.gz k=31 ref=~/data/reference/unwanted_seqs/RNACentral_ncRNAs.fasta outm=All_Cnidaria/Cf/detected_ncRNAs.fastq.gz stats=All_Cnidaria/Cf_stats.txt

# C. yamaguchii
cd ~/data/RNA-seq/2-rRNA_filter/; bbduk.sh in1=~/data/RNA-seq/1-adapters_trimmed/Cy/Cy1_trim.fastq.gz in2=~/data/RNA-seq/1-adapters_trimmed/Cy/Cy2_trim.fastq.gz out1=All_Cnidaria/Cy/clean_Cy1.fastq.gz out2=All_Cnidaria/Cy/clean_Cy2.fastq.gz k=31 ref=~/data/reference/unwanted_seqs/RNACentral_ncRNAs.fasta outm=All_Cnidaria/Cy/detected_ncRNAs.fastq.gz stats=All_Cnidaria/Cy_stats.txt

    # Cubozoan only search

# C. fleckeri
cd ~/data/RNA-seq/2-rRNA_filter/; bbduk.sh in1=~/data/RNA-seq/1-adapters_trimmed/Cf/Cf1_trim.fastq.gz in2=~/data/RNA-seq/1-adapters_trimmed/Cf/Cf2_trim.fastq.gz out1=Cubozoa_only/Cf/clean_Cf1.fastq.gz out2=Cubozoa_only/Cf/clean_Cf2.fastq.gz k=31 ref=~/data/reference/unwanted_seqs/Cubozoa_only.fasta.gz outm=Cubozoa_only/Cf/detected_ncRNAs.fastq.gz stats=Cubozoa_only/Cf_stats.txt

# C. yamaguchii
cd ~/data/RNA-seq/2-rRNA_filter/; bbduk.sh in1=~/data/RNA-seq/1-adapters_trimmed/Cy/Cy1_trim.fastq.gz in2=~/data/RNA-seq/1-adapters_trimmed/Cy/Cy2_trim.fastq.gz out1=Cubozoa_only/Cy/clean_Cy1.fastq.gz out2=Cubozoa_only/Cy/clean_Cy2.fastq.gz k=31 ref=~/data/reference/unwanted_seqs/Cubozoa_only.fasta.gz outm=Cubozoa_only/Cy/detected_ncRNAs.fastq.gz stats=Cubozoa_only/Cy_stats.txt

    # Only Chironex

# C. fleckeri
cd ~/data/RNA-seq/2-rRNA_filter/; bbduk.sh in1=~/data/RNA-seq/1-adapters_trimmed/Cf/Cf1_trim.fastq.gz in2=~/data/RNA-seq/1-adapters_trimmed/Cf/Cf2_trim.fastq.gz out1=Chironex_only/Cf/clean_Cf1.fastq.gz out2=Chironex_only/Cf/clean_Cf2.fastq.gz k=31 ref=~/data/reference/unwanted_seqs/Chironex_only.fasta outm=Chironex_only/Cf/detected_ncRNAs.fastq.gz stats=Chironex_only/Cf_stats.txt

# C. yamaguchii
cd ~/data/RNA-seq/2-rRNA_filter/; bbduk.sh in1=~/data/RNA-seq/1-adapters_trimmed/Cy/Cy1_trim.fastq.gz in2=~/data/RNA-seq/1-adapters_trimmed/Cy/Cy2_trim.fastq.gz out1=Chironex_only/Cy/clean_Cy1.fastq.gz out2=Chironex_only/Cy/clean_Cy2.fastq.gz k=31 ref=~/data/reference/unwanted_seqs/Chironex_only.fasta outm=Chironex_only/Cy/detected_ncRNAs.fastq.gz stats=Chironex_only/Cy_stats.txt

In [ ]:
# Fastqc analysis of rRNA-cleaned sequences
    # All Cnidarian-filtered
cd ~/data/shannc/RNA-seq/2-rRNA_filter/All_Cnidaria; fastqc $(find -name "*clean*" | find -name "*.fastq.gz") -o ~/data/RNA-seq/2-rRNA-filter --extract

#### Filtering ncRNA with `sortmerna`

In [ ]:
    # All Cnidarian search

# C. fleckeri
cd ~/data/RNA-seq/2-rRNA_filter/; sortmerna --reads ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf1_trim.fastq.gz --reads ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf2_trim.fastq.gz --workdir All_Cnidaria/Cf/sortmerna --ref ~/data/reference/unwanted_seqs/RNACentral_ncRNAs.fasta

# C. yamaguchii
cd ~/data/RNA-seq/2-rRNA_filter/; sortmerna --reads ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy1_trim.fastq.gz --reads ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy2_trim.fastq.gz --workdir All_Cnidaria/Cy/sortmerna --ref ~/data/reference/unwanted_seqs/RNACentral_ncRNAs.fasta

    # Cubozoan only search

# C. fleckeri
cd ~/data/RNA-seq/2-rRNA_filter/; sortmerna --reads ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf1_trim.fastq.gz --reads ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf2_trim.fastq.gz --workdir Cubozoa_only/Cf/sortmerna --ref ~/data/reference/unwanted_seqs/Cubozoa_only.fasta

# C. yamaguchii
cd ~/data/RNA-seq/2-rRNA_filter/; sortmerna --reads ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy1_trim.fastq.gz --reads ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy2_trim.fastq.gz --workdir Cubozoa_only/Cy/sortmerna --ref ~/data/reference/unwanted_seqs/Cubozoa_only.fasta

    # Chironex only

# C. fleckeri
cd ~/data/RNA-seq/2-rRNA_filter/; sortmerna --reads ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf1_trim.fastq.gz --reads ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf2_trim.fastq.gz --workdir Chironex_only/Cf/sortmerna --ref ~/data/reference/unwanted_seqs/Chironex_only.fasta

# C. yamaguchii
cd ~/data/RNA-seq/2-rRNA_filter/; sortmerna --reads ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy1_trim.fastq.gz --reads ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy2_trim.fastq.gz --workdir Chironex_only/Cy/sortmerna --ref ~/data/reference/unwanted_seqs/Chironex_only.fasta

#### Filtering prokaryotic RNA with `kraken2`
If you ever do get around to filtering for this, [this](https://academic-oup-com.ejournal.mahidol.ac.th/femsec/article/92/5/fiw064/2470076#61529333) paper provides information about the nature of the Cnidarian holobiont

#### Normalization with `ORNA`
A normalization step prior to assembly can reduce transcript redundancy, speeding up the assembly process and reducing the memory required. If you aren't concerned with memory, would this be necessary?  

- Flag info
    - `-kmer` the value of k for kmer size used in sorting. Devs recommend to using "the smallest k-mer" used in the debruijn graph assembler
    - `-base` Represents the base of the $\log$ function used to decide new k-mer abundance i.e. it determines the magnitude of the reduction. If the original k-mer abundance is 1000, and you choose base 10, the new abundance will be $\log_{10} 1000 = 3$. 
        - Thus, higher base value, higher reduction. In their original analysis, base 1.7 was found to be a good compromise bewteen data reduction and little loss in assembly quality 

In [ ]:
# Normalization of rRNA-filtered data, in respective directories
    # Cf
ORNA -pair1 resources/RNA-seq/1-adapters_trimmed/Cf/Cf1_trim.fastq.gz -pair2 resources/RNA-seq/1-adapters_trimmed/Cf/Cf2_trim.fastq.gz  -output Cf_N -type fastq 
    # Cy
ORNA -pair1 ~/data/RNA-seq/2-rRNA_filter/Cy/clean_Cy1.fastq.gz -pair2 ~/data/RNA-seq/2-rRNA_filter/Cy/clean_Cy1.fastq.gz  -output Cy_N -type fastq

### Assembly 

#### RNA spades
- Flag info
    - `-k` specifies k-mer size to use

- Output info
- SPAdes stores all output files in <output_dir> , which is set by the user.
    - <output_dir>/corrected/ directory contains reads corrected by BayesHammer in *.fastq.gz files; if compression is disabled, reads are stored in uncompressed *.fastq files
    - <output_dir>/scaffolds.fasta contains resulting scaffolds (recommended for use as resulting sequences)
    - <output_dir>/contigs.fasta contains resulting contigs
    - <output_dir>/assembly_graph_with_scaffolds.gfa contains SPAdes assembly graph and scaffolds paths in GFA 1.0 format
    - <output_dir>/assembly_graph.fastg contains SPAdes assembly graph in FASTG format
    - <output_dir>/contigs.paths contains paths in the assembly graph corresponding to contigs.fasta (see details below)
    - <output_dir>/scaffolds.paths contains paths in the assembly graph corresponding to scaffolds.fasta (see details below)


In [ ]:
# Assembly with only rRNA-filtering + adapter cleaning
    # Cf
rnaspades.py  -1 ~/data/RNA-seq/2-rRNA_filter/Cf/clean_Cf1.fastq.gz -2 ~/data/RNA-seq/2-rRNA_filter/Cf/clean_Cf2.fastq.gz -o ~/data/RNA-seq/4-assembly/Cf/1-2 -k
    # Cy
rnaspades.py  -1 ~/data/RNA-seq/2-rRNA_filter/Cy/clean_Cy1.fastq.gz -2 ~/data/RNA-seq/2-rRNA_filter/Cy/clean_Cy2.fastq.gz -o ~/data/RNA-seq/4-assembly/Cf/1-2 -k

#### Transabyss

In [ ]:
# Assembly with only adapter cleaning
    # Cf
transabyss  --pe ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf1_trim.fastq.gz --pe ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf2_trim.fastq.gz --outdir ~/data/RNA-seq/4-assembly/Cf/1/transabyss --name 1-tabyss_Cf.fa

    # Cy
transabyss  --pe ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy1_trim.fastq.gz --pe ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy2_trim.fastq.gz --outdir ~/data/RNA-seq/4-assembly/Cy/1/transabyss --name 1-tabyss_Cy.fa

In [ ]:
# Assembly with rRNA filtering + adapter cleaning
    # Cf
transabyss  --pe ~/data/RNA-seq/2-rRNA_filter/Cf/clean_Cf1.fastq.gz --pe ~/data/RNA-seq/2-rRNA_filter/Cf/clean_Cf2.fastq.gz --outdir ~/data/RNA-seq/4-assembly/Cf/1-2/transabyss --name 1-2-tabyss_Cf.fa

    # Cy
transabyss  --pe ~/data/RNA-seq/2-rRNA_filter/Cy/clean_Cy1.fastq.gz --pe ~/data/RNA-seq/2-rRNA_filter/Cy/clean_Cy2.fastq.gz --outdir ~/data/RNA-seq/4-assembly/Cy/1-2/transabyss --name 1-2-tabyss_Cf.fa

#### RNA bloom

In [ ]:
# Assembly with only adapter cleaning
    # Cf
rnabloom  -left ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf1_trim.fastq.gz -right ~/data/RNA-seq/1-adapters_trimmed/Cf/Cf2_trim.fastq.gz -outdir ~/data/RNA-seq/4-assembly/Cf/1/rnabloom -k 25

    # Cy
rnabloom  -left ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy1_trim.fastq.gz -right ~/data/RNA-seq/1-adapters_trimmed/Cy/Cy2_trim.fastq.gz -outdir ~/data/RNA-seq/4-assembly/Cy/1/rnabloom 

In [ ]:
# Assembly with ncRNA filtering 
    # Cf
rnabloom  -left ~/data/RNA-seq/2-rRNA_filter/Cf/clean_Cf1.fastq.gz -right ~/data/RNA-seq/2-rRNA_filter/Cf/clean_Cf2.fastq.gz -outdir ~/data/RNA-seq/4-assembly/Cf/1-2/rnabloom -k
    # Cy
rnabloom  -left ~/data/RNA-seq/2-rRNA_filter/Cy/clean_Cy1.fastq.gz -right ~/data/RNA-seq/2-rRNA_filter/Cy/clean_Cy2.fastq.gz -outdir ~/data/RNA-seq/4-assembly/Cy/1-2/rnabloom -k

#### Run this to get quick stats on assembly

In [ ]:
seqkit stats $(find /home/shannc/data/RNA-seq/4-assembly/ -maxdepth 3 -name *.fa) > /home/shannc/data/RNA-seq/4-assembly/assembly_stats.txt

### Post-assembly QC
`BUSCO`, `seqkit`, `DOGMA`

#### `BUSCO` & `DOGMA`
`BUSCO` assesses assembly quality by mapping the reads against highly conserved, well-expressed, orthologous genes that you would expect to find in good assembly. 

`DOGMA` has a similar function, but using protein domains instead

In [3]:
docker run -u $(id -u) -w /home/sc31/Bio_SDD/busco_QC/busco_wd -v /home/sc31/Bio_SDD/:/busco_QC ezlabgva/busco:v5.4.4_cv1 busco -i 2-rbloom_Cf.fa -l metazoa_odb10 -o 2-rbloom_Cf_results -m transcriptome 

docker run -u $(id -u) -w /home/sc31/bioinfo/busco_QC/busco_wd -v /home/sc31/bioinfo:/busco_QC ezlabgva/busco:v5.4.4_cv1 busco -i 1-rbloom_Cf.fa -l metazoa_odb10 -o 1-rbloom_Cf_results -m transcriptome 

SyntaxError: invalid syntax (2016770500.py, line 1)

#### `DETONATE`
The score obtained from `RSEM-EVAL` will help you decide which assembly is best. The short reads you pass to it should be the short reads passed to the assembler, so use the filtered reads.

##### Usage
- First argument specifies reads
- Second specifies assembly

In [ ]:
# Example evaluation for ncRNA-filtered C. fleckeri RNABloom assembly (2-rbloom_Cf.fa)
rsem-eval-caculate-score ~/data/ ~/data/RNA-seq/4-assembly/Cf/1-2/2-rbloom_Cf.fa  # Incomplete!!!

#### `transrate`


In [ ]:
# Check 1-2 Cf rbloom assembly
cd ~/data/RNA-seq/4-assembly/; conda activate transrate; transrate --assembly Cf/1-2/2-rbloom_Cf.fa --left ~/data/raw/Cf_RNA/Cflec_1.fastq --right ~/data/raw/Cf_RNA/Cflec_1.fastq; conda deactivate; cd transrate
# Check all Cy assemblies

### RNA classification

Since you already filtered out many ncRNAs in the pre-assembly quality control, you probably don't need to do additional sequence classification (i.e. separating out rRNAs).

### Sequence translation
Will use the ab initio predictor `TransDecoder` to obtain ORFs from the assembled transcripts

In [ ]:
cd /home/shannc/data/RNA-seq/6-Translated/; for assembly in $(find /home/shannc/data/RNA-seq/4-assembly/ -maxdepth 3 -name *.fa); do TransDecoder.LongOrfs -t $assembly; done